In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram 

In [33]:

def initialize(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    U_init = qc.to_gate()
    U_init.name = "$U_i$"
    return U_init

def diffuser(n):
    qc = QuantumCircuit(n)
    qubits = range(n)
    # Apply transformation |s> -> |00..0> (H-gates)
    qc.h(qubits)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    qc.x(qubits)
    # Do multi-controlled-Z gate
    qc.h(n-1)
    qc.mct(list(range(n-1)), n-1)  # multi-controlled-toffoli (cx)
    qc.h(n-1)
    # Apply transformation |11..1> -> |00..0>
    qc.x(qubits)
    # Apply transformation |00..0> -> |s>
    qc.h(qubits)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "$U_s$"
    return U_s

def XOR(qc, a, b, output):
    qc.cx(a, output)
    qc.cx(b, output)


def kakuro_oracle_1(qc, clause_qubits):
  
    xor_list = [[0, 1], [2, 3], [6, 7], [0, 2], [1, 6], [5, 7]]
    # Compute clauses
    i = 0
    for clause in xor_list:
        XOR(qc, clause[0], clause[1], clause_qubits[i])
        i += 1

    # "(x3 == 0) and"
    qc.x(3)
    qc.cx(3, clause_qubits[i])
    qc.x(3)
    i+=1
    # "(x4 == 1) and"
    qc.cx(4, clause_qubits[i])
    i+=1

    # Flip 'output' bit if all clauses are satisfied
    qc.mct(clause_qubits[0:i], output_qubit)

    # Uncompute clauses to reset clause-checking bits to 0
    i = 0
    for clause in xor_list:
        XOR(qc, clause[0], clause[1], clause_qubits[i])
        i += 1
    
    # "(x3 == 0) and"
    qc.x(3)
    qc.cx(3, clause_qubits[i])
    qc.x(3)
    i+=1
    # "(x4 == 1) and"
    qc.cx(4, clause_qubits[i])
    i+=1

    qc.barrier()

def chain_mcx(qc, controls, target, ancilla):
    assert(len(controls) - 2 == len(ancilla))
    l = len(controls)
    # my own implementation of mcx
    qc.barrier()
    qc.mcx(controls[0:2], ancilla[0])
    for i in range(2, l-1):
        qc.mcx([controls[i], ancilla[i-2]], ancilla[i-1])
    qc.mcx([controls[l-1], ancilla[l-3]], target)
    for i in range(l-2, 1, -1):
        qc.mcx([controls[i], ancilla[i-2]], ancilla[i-1]) 
    qc.mcx(controls[0:2], ancilla[0])   
    qc.barrier()

def kakuro_oracle_2(qc, clause_qubits):
    # "(x3 == 0) and"
    qc.x(3)
    qc.cx(3, clause_qubits[0])
    qc.x(3)
    # "(x4 == 1) and"
    qc.cx(4, clause_qubits[1])
    qc.barrier()
    snake = [3, 2, 0, 1, 6, 7, 5]
    # snake clause 1
    for i in range(len(snake)):
        if (i%2==0):
            qc.x(snake[i])
    ## 
    #qc.mcx(snake, clause_qubits[2])
    chain_mcx(qc, snake, clause_qubits[2], clause_qubits[3:8])
    for i in range(len(snake)):
        if (i%2==0):
            qc.x(snake[i])
    # snake clause 2
    qc.barrier()
    for i in range(len(snake)):
        if (i%2==1):
            qc.x(snake[i])
    #qc.mcx(snake, clause_qubits[2])
    chain_mcx(qc, snake, clause_qubits[2], clause_qubits[3:8])
    for i in range(len(snake)):
        if (i%2==1):
            qc.x(snake[i])
    qc.barrier()
    # Flip 'output' bit if all clauses are satisfied
    qc.mct(clause_qubits[0:3], output_qubit)
    # uncompute
    qc.barrier()
    for i in range(len(snake)):
        if (i%2==1):
            qc.x(snake[i])
    #qc.mcx(snake, clause_qubits[2])
    chain_mcx(qc, snake, clause_qubits[2], clause_qubits[3:8])
    for i in range(len(snake)):
        if (i%2==1):
            qc.x(snake[i])
    qc.barrier()
    for i in range(len(snake)):
        if (i%2==0):
            qc.x(snake[i])
    #qc.mcx(snake, clause_qubits[2])
    chain_mcx(qc, snake, clause_qubits[2], clause_qubits[3:8])
    for i in range(len(snake)):
        if (i%2==0):
            qc.x(snake[i])
    qc.barrier()
    qc.x(3)
    qc.cx(3, clause_qubits[0])
    qc.x(3)

    qc.cx(4, clause_qubits[1])
    # Compute clauses
    qc.barrier()
n=8
var_bits = QuantumRegister(n, name="x")
clause_qubits = QuantumRegister(8, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(n, name='cbits')
grover_circuit = QuantumCircuit(var_bits, clause_qubits, output_qubit, cbits)
grover_circuit.append(initialize(n), range(n))
# Initialize 'out0' in state |->
grover_circuit.initialize([1, -1]/np.sqrt(2), output_qubit)
grover_circuit.barrier()
for i in range(3):
    kakuro_oracle_2(grover_circuit, clause_qubits)
    grover_circuit.append(diffuser(n), range(n))
grover_circuit.measure(var_bits, cbits)
# grover_circuit.draw(output='mpl', filename='circuit.png', scale=0.3)

In [22]:
# the chain_mcx gate
qubits = QuantumRegister(7)
ancilla = QuantumRegister(5)
target = QuantumRegister(1)
classical = ClassicalRegister(13)
qc = QuantumCircuit(qubits, ancilla, target, classical)
qc.x(qubits[:-1])
chain_mcx(qc, qubits, target, ancilla)
qc.measure(list(qubits)+list(ancilla)+list(target), classical)
qc.draw()

┌───┐ ░                                                         ░ ┌─┐»
q633_0: ┤ X ├─░───■─────────────────────────────────────────────────■───░─┤M├»
        ├───┤ ░   │                                                 │   ░ └╥┘»
q633_1: ┤ X ├─░───■─────────────────────────────────────────────────■───░──╫─»
        ├───┤ ░   │                                                 │   ░  ║ »
q633_2: ┤ X ├─░───┼────■───────────────────────────────────────■────┼───░──╫─»
        ├───┤ ░   │    │                                       │    │   ░  ║ »
q633_3: ┤ X ├─░───┼────┼────■─────────────────────────────■────┼────┼───░──╫─»
        ├───┤ ░   │    │    │                             │    │    │   ░  ║ »
q633_4: ┤ X ├─░───┼────┼────┼────■───────────────────■────┼────┼────┼───░──╫─»
        ├───┤ ░   │    │    │    │                   │    │    │    │   ░  ║ »
q633_5: ┤ X ├─░───┼────┼────┼────┼────■─────────■────┼────┼────┼────┼───░──╫─»
        └───┘ ░   │    │    │    │    │         │    │    │    │    │   ░  ║ »
q633_6: ──────░───┼────┼────┼────┼────┼────■────┼────┼────┼────┼────┼───░──╫─»
              ░ ┌─┴─┐  │    │    │    │    │    │    │    │    │  ┌─┴─┐ ░  ║ »
q634_0: ──────░─┤ X ├──■────┼────┼────┼────┼────┼────┼────┼────■──┤ X ├─░──╫─»
              ░ └───┘┌─┴─┐  │    │    │    │    │    │    │  ┌─┴─┐└───┘ ░  ║ »
q634_1: ──────░──────┤ X ├──■────┼────┼────┼────┼────┼────■──┤ X ├──────░──╫─»
              ░      └───┘┌─┴─┐  │    │    │    │    │  ┌─┴─┐└───┘      ░  ║ »
q634_2: ──────░───────────┤ X ├──■────┼────┼────┼────■──┤ X ├───────────░──╫─»
              ░           └───┘┌─┴─┐  │    │    │  ┌─┴─┐└───┘           ░  ║ »
q634_3: ──────░────────────────┤ X ├──■────┼────■──┤ X ├────────────────░──╫─»
              ░                └───┘┌─┴─┐  │  ┌─┴─┐└───┘                ░  ║ »
q634_4: ──────░─────────────────────┤ X ├──■──┤ X ├─────────────────────░──╫─»
              ░                     └───┘┌─┴─┐└───┘                     ░  ║ »
  q635: ──────░──────────────────────────┤ X ├──────────────────────────░──╫─»
              ░                          └───┘                          ░  ║ »
 c6: 13/═══════════════════════════════════════════════════════════════════╩═»
                                                                           0 »
«                                            
«q633_0: ────────────────────────────────────
«        ┌─┐                                 
«q633_1: ┤M├─────────────────────────────────
«        └╥┘┌─┐                              
«q633_2: ─╫─┤M├──────────────────────────────
«         ║ └╥┘┌─┐                           
«q633_3: ─╫──╫─┤M├───────────────────────────
«         ║  ║ └╥┘┌─┐                        
«q633_4: ─╫──╫──╫─┤M├────────────────────────
«         ║  ║  ║ └╥┘┌─┐                     
«q633_5: ─╫──╫──╫──╫─┤M├─────────────────────
«         ║  ║  ║  ║ └╥┘┌─┐                  
«q633_6: ─╫──╫──╫──╫──╫─┤M├──────────────────
«         ║  ║  ║  ║  ║ └╥┘┌─┐               
«q634_0: ─╫──╫──╫──╫──╫──╫─┤M├───────────────
«         ║  ║  ║  ║  ║  ║ └╥┘┌─┐            
«q634_1: ─╫──╫──╫──╫──╫──╫──╫─┤M├────────────
«         ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
«q634_2: ─╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
«         ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
«q634_3: ─╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
«         ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
«q634_4: ─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
«         ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
«  q635: ─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
«         ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
« c6: 13/═╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
«         1  2  3  4  5  6  7  8  9  10 11 12

In [23]:
aer_simulator = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_simulator)
qobj = assemble(transpiled_qc)
result = aer_simulator.run(qobj).result()
print(result.get_counts())

{'0000000111111': 1024}


In [34]:
grover_circuit.draw()

┌────────┐            ░                 ░ ┌───┐ ░      »
    x_0: ───────────┤0       ├────────────░─────────────────░─┤ X ├─░──────»
                    │        │            ░                 ░ └───┘ ░      »
    x_1: ───────────┤1       ├────────────░─────────────────░───────░──────»
                    │        │            ░                 ░       ░      »
    x_2: ───────────┤2       ├────────────░─────────────────░───────░───■──»
                    │        │            ░ ┌───┐     ┌───┐ ░ ┌───┐ ░   │  »
    x_3: ───────────┤3       ├────────────░─┤ X ├──■──┤ X ├─░─┤ X ├─░───■──»
                    │  $U_i$ │            ░ └───┘  │  └───┘ ░ └───┘ ░   │  »
    x_4: ───────────┤4       ├────────────░───■────┼────────░───────░───┼──»
                    │        │            ░   │    │        ░ ┌───┐ ░   │  »
    x_5: ───────────┤5       ├────────────░───┼────┼────────░─┤ X ├─░───┼──»
                    │        │            ░   │    │        ░ ├───┤ ░   │  »
    x_6: ───────────┤6       ├────────────░───┼────┼────────░─┤ X ├─░───┼──»
                    │        │            ░   │    │        ░ └───┘ ░   │  »
    x_7: ───────────┤7       ├────────────░───┼────┼────────░───────░───┼──»
                    └────────┘            ░   │  ┌─┴─┐      ░       ░   │  »
    c_0: ─────────────────────────────────░───┼──┤ X ├──────░───────░───┼──»
                                          ░ ┌─┴─┐└───┘      ░       ░   │  »
    c_1: ─────────────────────────────────░─┤ X ├───────────░───────░───┼──»
                                          ░ └───┘           ░       ░   │  »
    c_2: ─────────────────────────────────░─────────────────░───────░───┼──»
                                          ░                 ░       ░ ┌─┴─┐»
    c_3: ─────────────────────────────────░─────────────────░───────░─┤ X ├»
                                          ░                 ░       ░ └───┘»
    c_4: ─────────────────────────────────░─────────────────░───────░──────»
                                          ░                 ░       ░      »
    c_5: ─────────────────────────────────░─────────────────░───────░──────»
                                          ░                 ░       ░      »
    c_6: ─────────────────────────────────░─────────────────░───────░──────»
                                          ░                 ░       ░      »
    c_7: ─────────────────────────────────░─────────────────░───────░──────»
         ┌──────────────────────────────┐ ░                 ░       ░      »
    out: ┤ Initialize(0.70711,-0.70711) ├─░─────────────────░───────░──────»
         └──────────────────────────────┘ ░                 ░       ░      »
cbits: 8/══════════════════════════════════════════════════════════════════»
                                                                           »
«                                                            ░ ┌───┐ ░       ░ »
«    x_0: ──■───────────────────────────────────────■────────░─┤ X ├─░───────░─»
«           │                                       │        ░ └───┘ ░ ┌───┐ ░ »
«    x_1: ──┼────■─────────────────────────────■────┼────────░───────░─┤ X ├─░─»
«           │    │                             │    │        ░       ░ ├───┤ ░ »
«    x_2: ──┼────┼─────────────────────────────┼────┼────■───░───────░─┤ X ├─░─»
«           │    │                             │    │    │   ░ ┌───┐ ░ └───┘ ░ »
«    x_3: ──┼────┼─────────────────────────────┼────┼────■───░─┤ X ├─░───────░─»
«           │    │                             │    │    │   ░ └───┘ ░       ░ »
«    x_4: ──┼────┼─────────────────────────────┼────┼────┼───░───────░───────░─»
«           │    │                             │    │    │   ░ ┌───┐ ░       ░ »
«    x_5: ──┼────┼──────────────■──────────────┼────┼────┼───░─┤ X ├─░───────░─»
«           │    │              │              │    │    │   ░ ├───┤ ░       ░ »
«    x_6: ──┼────┼────■─────────┼─────────■────┼────┼────┼───░─┤ X ├─░───────░─»
«           │    │    │         │    

In [35]:
aer_simulator = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(grover_circuit, aer_simulator)
qobj = assemble(transpiled_qc)
result = aer_simulator.run(qobj).result()
print(result.get_counts())

{'10100010': 3, '10100111': 5, '01001000': 6, '10111111': 1, '01010011': 7, '11010001': 5, '10010110': 176, '00100010': 1, '00111110': 3, '00101101': 4, '00110011': 7, '11011011': 4, '00000100': 3, '11100001': 4, '11001100': 2, '10100100': 3, '11110111': 3, '10100001': 6, '00001000': 4, '00101010': 3, '00110110': 2, '11001111': 2, '11100100': 4, '10110100': 5, '01011010': 5, '01100101': 3, '00111010': 1, '00100110': 5, '10010111': 3, '00001110': 2, '10010100': 4, '01110110': 7, '01001100': 5, '00011011': 9, '01101001': 7, '00010101': 4, '11110000': 4, '01101011': 2, '10100101': 2, '00110000': 2, '10011111': 2, '11011010': 3, '00000111': 8, '01000101': 4, '00001001': 3, '10100000': 4, '10110010': 4, '11001001': 5, '11011000': 6, '11111111': 3, '01100001': 3, '00000010': 4, '01000100': 3, '01111010': 3, '01010001': 4, '10111101': 3, '11111000': 2, '01000110': 2, '01111100': 3, '00100100': 4, '00111100': 4, '11000011': 2, '11101010': 6, '00001101': 4, '10000001': 8, '00111111': 1, '001000

In [36]:
result.get_counts()['10010110']

176

In [37]:
# prepare submission and check score
n = 8
var_bits = QuantumRegister(n, name="x")
clause_qubits = QuantumRegister(n, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(n, name='cbits')
grover_circuit = QuantumCircuit(var_bits, clause_qubits, output_qubit, cbits)
kakuro_oracle_2(grover_circuit, clause_qubits)
qc_transpiled = transpile(grover_circuit, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())
qc_transpiled.qasm(filename='kakuro2.qasm')

465
OrderedDict([('u', 400), ('cx', 282), ('barrier', 15)])


'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg x[8];\nqreg c[8];\nqreg out[1];\ncreg cbits[8];\nu(pi,0,pi) x[3];\ncx x[3],c[0];\nu(pi,0,pi) x[3];\ncx x[4],c[1];\nbarrier x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],c[0],c[1],c[2],c[3],c[4],c[5],c[6],c[7],out[0];\nu(pi,0,pi) x[0];\nu(pi,0,pi) x[3];\nu(pi,0,pi) x[5];\nu(pi,0,pi) x[6];\nbarrier x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],c[0],c[1],c[2],c[3],c[4],c[5],c[6],c[7],out[0];\nu(pi/2,0,pi) c[2];\nu(pi/2,0,pi) c[3];\nu(pi/2,0,pi) c[4];\nu(pi/2,0,pi) c[5];\nu(pi/2,0,pi) c[6];\nu(pi/2,0,pi) c[7];\ncx x[2],c[3];\nu(0,0,-pi/4) c[3];\ncx x[3],c[3];\nu(0,0,pi/4) c[3];\ncx x[2],c[3];\nu(0,0,-pi/4) c[3];\nu(0,0,pi/4) x[2];\ncx x[3],c[3];\nu(pi/2,0,-3*pi/4) c[3];\ncx c[3],c[4];\nu(0,0,-pi/4) c[4];\ncx x[0],c[4];\nu(0,0,pi/4) c[4];\ncx c[3],c[4];\nu(0,0,pi/4) c[3];\nu(0,0,-pi/4) c[4];\ncx x[0],c[4];\nu(pi/2,0,-3*pi/4) c[4];\ncx c[4],c[5];\nu(0,0,-pi/4) c[5];\ncx x[0],c[3];\nu(0,0,-pi/4) c[3];\nu(0,0,pi/4) x[0];\ncx x[0],c[3];\ncx x[1],c[5];\nu(0,0,pi/4